# Models + Admin

In de `models.py` file beschrijven we het data-model van onze app.

## Database setup

Eerst kijken we naar de `settings.py` file.

Hierin vinden we de settings van de database terug.

In [ ]:
# Database
# https://docs.djangoproject.com/en/4.2/ref/settings/#databases

DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.sqlite3',
        'NAME': BASE_DIR / 'db.sqlite3',
    }
}


We zien dat Django standaard gebruik maakt van "SQLite3".

Deze database is perfect om mee te werken tijdens de development fase.

Wanneer je niet "SQLite" gebruikt voegen we naast `ENGINE` en `NAME` ook nog `USER`, `PASSWORD` & `HOST` toe. Maar hier komen we later op terug. Voorlopig is "SQLite" dus voldoende om mee te werken.

In de `settings.py` vind je bovenaan ook de `INSTALLED_APPS` terug. Hier zie je nu de standaard apps die door Django worden gebruikt.

In [ ]:
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
]

* django.contrib.admin          – The admin site.
* django.contrib.auth           – An authentication system.
* django.contrib.contenttypes   – A framework for content types.
* django.contrib.sessions       – A session framework.
* django.contrib.messages       – A messaging framework.
* django.contrib.staticfiles    – A framework for managing static files.

Sommige van deze apps maken gebruik van database tabellen.

We hebben onze database echter nog niet geïntialiseerd. 

We kunnen checken welke database acties er reeds zijn uitgevoerd. Of staan te wachten.

In [ ]:
python manage.py showmigrations

We kunnen deze migraties uitvoeren door:

In [ ]:
python manage.py migrate

We kunnen nu een sqlite database viewer / editor downloaden.  (https://sqlitebrowser.org)

En nu kunnen we kijken welke tabellen en velden er allemaal zijn aangemaakt/

## Opmaken Polls model

In onze `models.py`-file definiëren we nu de data die we willen gebruiken.

We schrijven:


In [ ]:
from django.db import models

# Create your models here.
class Question(models.Model):
    question_text = models.CharField(max_length=200)
    pub_date = models.DateTimeField("date published")

class Choice(models.Model):
    question = models.ForeignKey(Question, on_delete=models.CASCADE)
    choice_text = models.CharField(max_length=200)
    votes = models.IntegerField(default=0)

Elke class zal worden omgezet naar een tabel in de database.

Binnen elk van deze klassen definiëren we de velden (kolommen) die deel van de tabel zullen uitmaken.
De naam van het veld (bv. question_text) is de naam die wij binnen in de python code zullen gebruiken. Het is ook de kolom-naam in de database.

<Kort uitleggen wat de verschillende velden beteken, alsook de argumenten>

https://docs.djangoproject.com/en/4.2/ref/models/fields/

als we `python manage.py makemigrations` runnen, gebeurt er nog niets.

We moeten eerst onze app toevoegen aan de `INSTALLED_APPS` in onze `settings.py`.

we doen dit door het volgende `"polls.apps.PollsConfig",` toe te voegen.

Wanneer we nu `python manage.py makemigrations` uitvoeren, dan zullen we zien dat ons model wordt klaar gezet voor de migratie.

dit commando zal zoeken naar alle mogelijke veranderingen. Wanneer je echter alleen de "polls" veranderingen wil uitvoeren kan je ook `python manage.py makemigrations polls` runnen.

Als we nu onze DB viewer terug openen, zien we dat de polls tabellen ook terugvinden.
Nu zien we dat er naast de aangemaakte velden ook automatisch een id veld wordt aangemaakt, de primary key. (dit is een auto-increment veld)

Voor diegenen die graag de SQL code willen zien die wordt gebruikt bij deze migratie kan het volgende commando uitvoeren `python manage.py sqlmigrate polls 0001`. 

We zien in de folder `migrations` de file `0001_initial.py` die deze query aanmaakt.

De exacte SQL-query syntax zal afhangen van de database die je in je settings hebt gedefinieerd.

je kan ook nog `python manage.py check` gebruiken om te weten te kopen of er issues zijn met potentiële migraties. Zonder een migratie te hoeen uitvoeren of veranderingen in de database uit te voeren.

Django biedt je dus de mogelijkheid om je modellen aan te passen en door te voeren in je database. Dit zonder verlies aan data. 

Belangrijk is de volgende 3 stappen te onthouden:
1. aanpassen model
2. run `python manage.py makemigrations`
3. run `python manage.py migrate`


## Django Admin

Via de Django admin geef je klant de mogelijkheid om op een gebruiksvriendelijke manier de data van applicatie te zien en aan te passen.

Als we onze server opnieuw opstarten `python manage.py runserver` en we gaan naar `http://127.0.0.1:8000/admin/` zien we de login voor deze ADMIN.

We hebben natuurlijk nog geen users aangemaakt, dus daarom moeten we eerst het volgende command runnen.
`python manage.py createsuperuser`

als username: `admin`
pasword: `admin`

(we gebruiken dit "paswoord" enkel in development. In productie zorg je best voor een vele sterker.)

Nu kunnen we inloggen op onze admin site.

We zien `Groups` & `Users`, maar nog niet onze `Questions` & `Choices`.

Daarvoor moeten we in de file `admin.py` de volgende code toevoegen:

In [ ]:
from django.contrib import admin

from .models import Question, Choice

admin.site.register(Question)
admin.site.register(Choice)

We kunnen nu een Question toevoegen. via onze admin-site.

Question:
    question_text = "What's up?"
    pub_date = "now"

We zien nu in de admin-site dat er Question is. Maar de representatie is niet echt heel leesbaar.

We voegen bij de Model-klass een "__str__" methode toe.

In [ ]:
def __str__(self):
    return self.question_text

We zien nu ook dat in onze admin-site de "Question(1)" vervangen is door de "question_text".


We doen het zelfde met de Choice - klasse

In [ ]:
def __str__(self) -> str:
    return self.choice_text